In [77]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

In [2]:
# Loading clustered dataset
file = "Resources/MLM/clustered_data.csv"
clustered_df = pd.read_csv(file, index_col=0)
clustered_df.head(10)

,abortion_status,total_community_health_centers,uninsured,total_insured,maternal_mortality,population,land_area_sqmi,no_doctor_visits,mammogram,no_provider,...,abortions_occurring_state,abortions_residence_state,no_services,few_services,restricted_services,full_service,PC 1,PC 2,PC 3,Class
state,,,,,,,,,,,,,,,,,,,,,
Alabama,Abortion Ban In Effect,17,165700,1462900,109.285826,4767100,50645,0.12,0.74,0.13,...,5700,9060,8,44,5,0,-3.441609,1.226503,0.812852,0
Alaska,Abortion Available,27,29200,207200,84.486218,701700,570641,0.12,0.63,0.23,...,1240,1320,3,7,0,4,-2.944785,0.679185,3.155152,0
Arizona,Status of pre-Roe ban unclear,23,309000,2132900,87.072920,7098000,113594,0.11,0.68,0.19,...,13320,13820,19,28,7,0,-1.713550,1.250492,0.170481,0
Arkansas,Abortion Ban In Effect,12,83800,865900,124.819154,2922500,52035,0.12,0.70,0.13,...,3250,4510,9,31,3,0,-3.252214,0.370151,1.473720,0
California,Abortion Available,175,1034300,11885100,32.123048,38642700,155779,0.09,0.68,0.19,...,154060,152400,51,97,0,124,10.480858,9.234399,-1.186718,2
Colorado,Abortion Available,19,204300,1758600,43.906723,5611800,103642,0.11,0.67,0.17,...,13420,11830,17,34,0,15,-0.319058,-0.121292,-0.656476,1
Connecticut,Abortion Available,16,58100,1038000,53.795577,3453300,4842,0.07,0.78,0.10,...,11170,11460,12,8,0,18,3.635465,-3.194741,-0.861823,4
Delaware,Abortion Available,3,24800,295800,48.113934,940300,1949,0.09,0.74,0.12,...,1830,2870,10,0,0,2,2.135973,-3.430561,0.971375,4
District of Columbia,Abortion Available,8,7600,251600,78.882128,671300,61,0.06,0.74,0.15,...,9410,5010,2,0,0,2,2.323861,-3.451662,0.935299,4


In [3]:
clustered_df.dtypes

abortion_status                               object
total_community_health_centers                 int64
uninsured                                      int64
total_insured                                  int64
maternal_mortality                           float64
population                                     int64
land_area_sqmi                                 int64
no_doctor_visits                             float64
mammogram                                    float64
no_provider                                  float64
pap_smear                                    float64
prescription_contraception                    object
otc_methods                                   object
male_sterilization                            object
female_sterilization                          object
cost_sharing                                  object
teen_births                                  float64
poverty_under_200                            float64
percent_of_all_us_abortions                  f

In [217]:
#chc_df = pd.get_dummies(data = clustered_df.drop(columns=['PC 1','PC 2','PC 3']),
#                        columns=['abortion_status','prescription_contraception',
#                              'otc_methods','male_sterilization','female_sterilization','cost_sharing'])
#chc_df

In [211]:
#chc_df = clustered_df[['total_community_health_centers','population','land_area_sqmi','Class']]
#chc_df

In [241]:
chc_df = clustered_df[['PC 1','PC 2','PC 3','Class','total_community_health_centers']]

In [253]:
#training_df = chc_df[(chc_df['Class']==4) | (chc_df['Class']==2)]
training_df = chc_df[chc_df['Class']==4]
X_train = training_df.drop(columns='total_community_health_centers')
y_train = training_df['total_community_health_centers']

In [254]:
fit_df = chc_df
X = fit_df.drop(columns=['total_community_health_centers'])
y = fit_df[['total_community_health_centers']]

In [255]:
# Creating an object of LinearRegression class
LR = LinearRegression()

# Fitting the training data
LR.fit(X_train, y_train)

LinearRegression()

In [256]:
y_pred = LR.predict(X)
y_pred

array([  4.53034111,   0.5940824 ,  13.57849949,   0.79246278,
       103.78403044,  15.7528049 ,  21.60642505,  10.71763079,
        11.54783421,  49.61742224,  26.31366037,   7.48696472,
        -0.50830282,  41.42579069,  12.37711471,  10.52977368,
         7.59557108,   4.21112321,   5.88820412,  13.45085044,
        24.4229069 ,  26.48894788,  29.23214748,  15.93479607,
         1.56685441,   7.89718484,   4.27121767,   3.06321646,
        12.82477982,  10.05505812,  34.83283718,  11.46223334,
        65.57405888,  26.38776349,  -1.73806303,  23.60101575,
         3.85682814,  19.31337154,  32.21765473,   8.03030056,
         6.08161487,  -3.2152145 ,  10.03398913,  48.70550236,
         0.70986878,  10.16830041,  22.46738164,  24.2707101 ,
         0.45466499,  12.64314161,  -7.15699263])

In [257]:
compare_df = y['total_community_health_centers'].to_frame()
compare_df['predictions'] = y_pred.astype(int)
compare_df['addl_chc_needed'] = compare_df['predictions'] - compare_df['total_community_health_centers']
compare_df['Class'] = chc_df['Class']
#compare_df = compare_df[compare_df['addl_chc_needed']>0]
compare_df.sort_values(by=['addl_chc_needed'],ascending=False)

,total_community_health_centers,predictions,addl_chc_needed,Class
state,,,,
New Jersey,23,34,11,4
Maryland,17,24,7,4
Delaware,3,10,7,4
Connecticut,16,21,5,4
Nevada,8,12,4,4
District of Columbia,8,11,3,4
New York,63,65,2,2
Florida,47,49,2,3
Rhode Island,8,8,0,1
